In [6]:
import pandas as pd
import spacy
import nltk
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

# Load NLP Model
nlp = spacy.load("en_core_web_sm")
nltk.download("punkt")

# Load Dataset
df = pd.read_csv("history_intent.csv")

# Split into train & test
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["intent"], test_size=0.2, random_state=42)

# TF-IDF Vectorizer + SVM Model
model = make_pipeline(TfidfVectorizer(), SVC(kernel="linear"))
model.fit(X_train, y_train)

# Save the model
joblib.dump(model, "intent_classifier.pkl")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aspire_Lays\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['intent_classifier.pkl']

In [7]:
import joblib
import random

# Load Intent Classifier
intent_model = joblib.load("intent_classifier.pkl")

# Knowledge Base
HISTORY_RESPONSES = {
    "history_event": {
        "world war 2": "World War II lasted from 1939 to 1945, involving major world powers like the USA, Germany, and the Soviet Union.",
        "cold war": "The Cold War (1947-1991) was a geopolitical tension between the USA and USSR.",
        "french revolution": "The French Revolution (1789-1799) led to radical social and political change in France."
    },
    "example_request": "Here's an example: The fall of the Roman Empire in 476 AD led to the Middle Ages.",
    "history_summary": "History is full of key moments, like the Industrial Revolution and the rise of democracy.",
    "historical_figure": "Napoleon was a French military leader who rose to power during the French Revolution."
}

# Generate a Response
def generate_response(user_input):
    """ Classify intent and generate a response """
    intent = intent_model.predict([user_input])[0]

    if intent == "history_event":
        for key in HISTORY_RESPONSES["history_event"]:
            if key in user_input.lower():
                return HISTORY_RESPONSES["history_event"][key]
        return "I can discuss historical events like World War 2, the Cold War, or the French Revolution."

    return HISTORY_RESPONSES.get(intent, "I'm not sure, but I can try to help!")


In [10]:
def analyze_discourse(user_input):
    """ Perform discourse analysis with Named Entity Recognition (NER) """
    doc = nlp(user_input)

    # Extract named entities (historical events, persons, dates)
    entities = [ent.text for ent in doc.ents if ent.label_ in ("EVENT", "DATE", "PERSON", "ORG")]

    # Extract subjects and objects based on dependency parsing
    subjects = [token.text for token in doc if token.dep_ in ("nsubj", "nsubjpass")]
    objects = [token.text for token in doc if token.dep_ in ("dobj", "pobj")]

    # If named entities are found, they should be considered as subjects
    if entities:
        subjects.extend(entities)

    return {"subjects": list(set(subjects)), "objects": list(set(objects))}


In [12]:
print(analyze_discourse("Tell me about World War 2 and its impact."))


print(analyze_discourse("Explain the causes of the French Revolution."))


print(analyze_discourse("Who was Napoleon?"))



{'subjects': ['World War 2'], 'objects': ['me', 'War']}
{'subjects': ['the French Revolution'], 'objects': ['causes', 'Revolution']}
{'subjects': ['Napoleon'], 'objects': []}
